In [100]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import sharppy.plot.skew as skew
from sharppy.sharptab import winds, utils, params, thermo, interp, profile
from sharppy.io.spc_decoder import SPCDecoder
import xarray as xr
from metpy.calc import dewpoint_from_relative_humidity
import glob
from matplotlib import gridspec
from matplotlib.ticker import ScalarFormatter, MultipleLocator
import pyart
import cartopy.crs as ccrs
import warnings; warnings.simplefilter("ignore")
import zipfile
from IPython.display import display, Markdown
import netCDF4 as nc
from sklearn.cluster import KMeans as kmeans
import seaborn as sns
import tqdm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes as inset
from matplotlib.patches import Circle
from metpy.calc import storm_relative_helicity, bunkers_storm_motion
from metpy.units import units

In [132]:
def get_sounding(time, lat, lon, var_max):

    str_fmt = "/g/data/rt52/era5/pressure-levels/reanalysis/%s/%d/%s_era5_oper_pl_%s*.nc" 
    str_fmt_sfc = "/g/data/rt52/era5/single-levels/reanalysis/%s/%d/%s_era5_oper_sfc_%s*.nc"
    
    # m, latlon = get_point_data(time,lat,lon,50,[var_max])
    # print(m)
    # lat = latlon[0]
    # lon = latlon[1]
    
    z=xr.open_dataset(glob.glob(str_fmt % ("z",time.year,"z",time.strftime("%Y%m01")))[0])["z"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")
    u=xr.open_dataset(glob.glob(str_fmt % ("u",time.year,"u",time.strftime("%Y%m01")))[0])["u"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")
    v=xr.open_dataset(glob.glob(str_fmt % ("v",time.year,"v",time.strftime("%Y%m01")))[0])["v"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")
    
    us=xr.open_dataset(glob.glob(str_fmt_sfc % ("10u",time.year,"10u",time.strftime("%Y%m01")))[0])["u10"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")    
    vs=xr.open_dataset(glob.glob(str_fmt_sfc % ("10v",time.year,"10v",time.strftime("%Y%m01")))[0])["v10"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")    
    zs=xr.open_dataset(glob.glob(str_fmt_sfc % ("z",time.year,"z",time.strftime("%Y%m01")))[0])["z"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest") 
    ps=xr.open_dataset(glob.glob(str_fmt_sfc % ("sp",time.year,"sp",time.strftime("%Y%m01")))[0])["sp"].\
            sel({"time":time.replace(minute=0)}).interp({"longitude":lon, "latitude":lat},method="nearest")    
    
    p=z.level
    prof_pres = np.flip(p.values.squeeze())
    prof_hgt = np.flip(z.values.squeeze()/9.8)
    prof_u = np.flip(u.values.squeeze()*1.94)
    prof_v = np.flip(v.values.squeeze()*1.94)
    agl_inds = prof_pres <= (ps.values/100.)
    
    pres=np.insert(prof_pres[agl_inds], 0, ps.values/100.)
    hght=np.insert(prof_hgt[agl_inds], 0, zs.values/9.8)
    u=np.insert(prof_u[agl_inds], 0, us.values*1.94)
    v=np.insert(prof_v[agl_inds], 0, vs.values*1.94)
    
    f = xr.open_mfdataset("/g/data/eg3/ab4502/ExtremeWind/aus/era5/era5_"+time.strftime("%Y%m01")+"*.nc")[var_max].\
            sel({"time":time.replace(minute=0)}).interp({"lon":lon, "lat":lat},method="nearest")
    print(f.values)
    
    return u, v, hght, pres
    
# def get_point_data(time,lat,lon,r,var,plot=False,vmin=None,vmax=None):
    
#     f = xr.open_dataset(glob.glob("/g/data/eg3/ab4502/ExtremeWind/aus/era5/era5_"\
#                             +time.strftime("%Y%m")+"*.nc")[0])[var+["mu_cape"]].sel({"time":time.replace(minute=0)})
    
#     lats = f.coords.get("lat").values
#     lons = f.coords.get("lon").values
#     x,y = np.meshgrid(lons,lats)
#     dist_km = (latlon_dist(lat, lon, y, x) )
#     mask = get_mask(lons,lats)
#     a,b = np.where( (dist_km <= r) & (mask == 1) )
#     target_lons = xr.DataArray(lons[b],dims="points")
#     target_lats = xr.DataArray(lats[a],dims="points")    
#     f_slice = (f.sel(lon=target_lons, lat=target_lats))
    
#     if plot:
#         plt.figure()
#         ax=plt.axes(projection=ccrs.PlateCarree())
#         temp = np.where((dist_km <= r) & (mask == 1), f.values, np.nan)
#         c=ax.pcolormesh(x,y,temp,vmin=vmin,vmax=vmax)
#         plt.colorbar(c)
#         ax.coastlines()
    
#     #Return the value of the point with the highest absolute value
#     return pd.DataFrame([f_slice[v].values[np.abs(f_slice[v]).argmax()] for v in var], index=var),\
#             [f_slice.lat.values[np.abs(f_slice[var[0]]).argmax()],\
#              f_slice.lon.values[np.abs(f_slice[var[0]]).argmax()] ]

# def latlon_dist(lat, lon, lats, lons):

#         #Calculate great circle distance (Harversine) between a lat lon point (lat, lon) and a list of lat lon
#         # points (lats, lons)
                        
#         R = 6373.0
                        
#         lat1 = np.deg2rad(lat)
#         lon1 = np.deg2rad(lon)
#         lat2 = np.deg2rad(lats)
#         lon2 = np.deg2rad(lons)
                
#         dlon = lon2 - lon1
#         dlat = lat2 - lat1

#         a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
#         c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

#         return (R * c)

# def get_mask(lon,lat,thresh=0.5):
#     #Return lsm for a given domain (with lats=lat and lons=lon)
#     lsm,nat_lon,nat_lat = get_lsm()
#     lon_ind = np.where((nat_lon >= lon[0]) & (nat_lon <= lon[-1]))[0]
#     lat_ind = np.where((nat_lat >= lat[-1]) & (nat_lat <= lat[0]))[0]
#     lsm_domain = lsm[(lat_ind[0]):(lat_ind[-1]+1),(lon_ind[0]):(lon_ind[-1]+1)]
#     lsm_domain = np.where(lsm_domain > thresh, 1, 0)

#     return lsm_domain

# def get_lsm():
#     #Load the ERA5 land-sea fraction
#     lsm_file = nc.Dataset("/g/data/rt52/era5/single-levels/reanalysis/lsm/1979/lsm_era5_oper_sfc_19790101-19790131.nc")
#     lsm = np.squeeze(lsm_file.variables["lsm"][0])
#     lsm_lon = np.squeeze(lsm_file.variables["longitude"][:])
#     lsm_lat = np.squeeze(lsm_file.variables["latitude"][:])
#     lsm_file.close()
#     return [lsm,lsm_lon,lsm_lat]    


def srh_test(u, v, z, p, d):
    rm, lm, m06 = bunkers_storm_motion(p*units.hectopascal,u*units.meter_per_second,v*units.meter_per_second,z*units.meter)
    print(storm_relative_helicity(z*units.meter, u*units.meter_per_second, v*units.meter_per_second, d*units.meter, bottom=0*units.meter,
                       storm_u=lm[0],storm_v=lm[1]))

In [133]:
u, v, z, p = get_sounding(dt.datetime(2010,12,16,3,25),-27.6297,152.7111,"srh06_left")

176.23438


In [134]:
srh_test(u,v,z,p,6000)

(<Quantity(10.9408008, 'meter ** 2 / second ** 2')>, <Quantity(-427.423104, 'meter ** 2 / second ** 2')>, <Quantity(-416.482303, 'meter ** 2 / second ** 2')>)


(<Quantity(10.9408008, 'meter ** 2 / second ** 2')>, <Quantity(-427.423104, 'meter ** 2 / second ** 2')>, <Quantity(-416.482303, 'meter ** 2 / second ** 2')>)


In [80]:
get_srh(u[np.newaxis,np.newaxis].T,v[np.newaxis,np.newaxis].T,z[np.newaxis,np.newaxis].T,0,6000,0)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [81]:
def get_srh():
    us6 = u6 - u0
    vs6 = v6 - v0
    tmp = 7.5 / (np.sqrt(np.square(us6) + np.square(vs6)))
    u_storm_right = umean06 + (tmp * vs6)
    v_storm_right = vmean06 - (tmp * us6)
    u_storm_left = umean06 - (tmp * vs6)
    v_storm_left = vmean06 + (tmp * us6)
    hgt = group.z.values - terrain
    u = group.ua.values
    v = group.va.values
    
    hgt_bot=0
    hgt_top=6000
    u_ma = np.ma.masked_where(np.squeeze((hgt < hgt_bot) | (hgt > hgt_top) | (np.isnan(u)) | \
            (np.isnan(hgt_bot)) | (np.isnan(hgt_top)) | (np.isnan(hgt))), u)
    v_ma = np.ma.masked_where(np.squeeze((hgt < hgt_bot) | (hgt > hgt_top) | (np.isnan(v)) | \
            (np.isnan(hgt_bot)) | (np.isnan(hgt_top)) | (np.isnan(hgt))) , v)
    sru_left = u_ma - u_storm_left
    srv_left = v_ma - v_storm_left
    layers_left = (sru_left[1:] * srv_left[:-1]) - (sru_left[:-1] * srv_left[1:])
    srhe_left = abs(np.sum(layers_left))

(37, 1, 1)

In [ ]:
details_list = [\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2006,9,24,1,41)], "rid":"Melbourne", "title":"Melbourne"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2009,11,20,3,6)], "rid":"Melbourne", "title":"Melbourne"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2010,3,6,3,30)], "rid":"Melbourne", "title":"Melbourne"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2010,6,17,0,25)], "rid":"Melbourne", "title":"Melbourne"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2011,11,18,7,4)], "rid":"Melbourne", "title":"Melbourne"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2011,12,25,6,45)], "rid":"Melbourne", "title":"Melbourne" },\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2012,2,26,9,11)], "rid":"Melbourne", "title":"Melbourne"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2013,3,21,3,45)], "rid":"Melbourne", "title":"Melbourne" },\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2015,2,28,9,25)], "rid":"Melbourne", "title":"Melbourne"},\
       \
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2010,8,2,15,43)], "rid":"Sydney", "title":"Sydney"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2012,8,10,4,21)], "rid":"Sydney", "title":"Sydney"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2013,2,23,13,43)], "rid":"Sydney", "title":"Sydney"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2014,10,14,11,39)], "rid":"Sydney", "title":"Sydney"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2015,3,1,4,34)], "rid":"Sydney", "title":"Sydney"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2015,12,16,2,3)], "rid":"Sydney", "title":"Sydney"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2016,1,14,4,20)], "rid":"Sydney", "title":"Sydney"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2016,6,4,3,42)], "rid":"Sydney", "title":"Sydney"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2017,4,9,8,3)], "rid":"Sydney", "title":"Sydney"},\
       \
        {"lat":-27.6297, "lon":152.7111, "time" : [dt.datetime(2010,12,16,3,25)], "rid":"Amberley", "title":"Amberley"},\
        {"lat":-27.6297, "lon":152.7111, "time" : [dt.datetime(2011,1,18,5,5)], "rid":"Amberley", "title":"Amberley"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2011,10,7,19,22)], "rid":"Amberley", "title":"Oakey"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2013,10,18,4,4)], "rid":"Amberley", "title":"Oakey"},\
        {"lat":-27.6297, "lon":152.7111, "time" : [dt.datetime(2013,11,23,9,45)], "rid":"Amberley", "title":"Amberley"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2014,1,23,4,52)], "rid":"Amberley", "title":"Oakey"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2016,1,29,4,22)], "rid":"Amberley", "title":"Oakey"},\
        {"lat":-27.6297, "lon":152.7111, "time" : [dt.datetime(2016,12,18,4,14)], "rid":"Amberley", "title":"Amberley"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2018,2,13,7,5)], "rid":"Amberley", "title":"Oakey"},\
       \
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2007,10,27,9,3)], "rid":"Woomera", "title":"Woomera"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2008,12,9,6,18)], "rid":"Woomera", "title":"Woomera"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2010,12,7,9,27)], "rid":"Woomera", "title":"Woomera"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2011,11,8,9,47)], "rid":"Woomera", "title":"Woomera"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2012,1,29,4,42)], "rid":"Woomera", "title":"Woomera"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2012,11,30,8,33)], "rid":"Woomera", "title":"Woomera"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2014,10,31,7,52)], "rid":"Woomera", "title":"Woomera"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2015,12,7,11,20)], "rid":"Woomera", "title":"Woomera"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2017,12,18,11,30)], "rid":"Woomera", "title":"Woomera"},\
         ]

profs = []
mupcls = []
for details in tqdm.tqdm(details_list):
    prof, mu_pcl = plot_sounding(details["time"][0],details["lat"],details["lon"], plot=False)
    profs.append(prof)
    mupcls.append(mu_pcl)